In [110]:
#This attempt assumed that subpackets are packets with the packet header, this is not the case and needed to be redone

#Returns popped value, remaining stream, and number of bits in remaining stream
def popLeftN(stream, size, n):
    #Pad if we need to pop more than size bits
    if n > size:
        print("Warning: Padding extra bits")
        stream << (n-size)
        size = n
    
    mask = 2**n -1
    shift = size - n
    leftN = (stream >> shift) & mask
    remainder = stream - (leftN << shift)
    return leftN, remainder, shift


def readPacketData(stream, numBits):
    #Get version
    version, stream, numBits = popLeftN(stream, numBits, 3)
    
    #Get typeID
    typeID, stream, numBits = popLeftN(stream, numBits, 3)
    
    number = 0
    lengthTypeID = -1
    #Check literal value
    if typeID == 4:
        foundEnd = False
        while (not foundEnd) and stream > 0:
            bits, stream, numBits = popLeftN(stream, numBits, 5)
            foundEnd = (bits >> 4) ^ 1
            number = (number << 4) | (bits & 0xF)
            #print("Have number {} and foundEnd is {} from bits {}".format(number, foundEnd, bits))
        if (not foundEnd) and stream != 0:
            print("Error: Did not find end of number. Please investigate")
            return 0, 0, 0, 0, 0, 0
    else:
        lengthTypeID, stream, numBits = popLeftN(stream, numBits, 1)
        
    #Check length type 0
    if lengthTypeID == 0:
        number, stream, numBits = popLeftN(stream, numBits, 15)
    #Check length type 1:
    if lengthTypeID ==1:
        number, stream, numBits = popLeftN(stream, numBits, 11)
        
    return version, typeID, lengthTypeID, number, stream, numBits

#Processes entirety of stream up to numBitsRemaining
#This is an add version operator
def processPacketsNumBits(streamOrig, numBitsOrig):
    versionTotal = 0
    streamRemaining = streamOrig
    numBitsRemaining = numBitsOrig
    minPacketSize = 11
    
    while numBitsRemaining > 0:
        if numBitsRemaining < minPacketSize:
            print("Warning: Ignoring the last extra {} bits of packet".format(numBitsRemaining))
            break
        
        #number is valid of the number or the return of number of bits or number of packets.
        #stream is the remaining stream after the header is read
        #numBits is the number of bits remaining of the original stream
        version, typeID, lengthType, number, streamRemaining, numBitsRemaining = readPacketData(streamRemaining, numBitsRemaining)        
        print("Read type numbits packet with version {}, typeID {}, lengthType {}, number {}, and remainingBits {}".format(version, typeID, lengthType, number, numBitsRemaining))
        versionTotal += version
        #if version == 0:
        #    print("version error, zero version is not allowed")
        #    return
        
        #Handle number
        if typeID ==4:
            continue
        #Handle operator where we know the number of bits of the packets
        else:
            if lengthType == 0:
                subStream = streamRemaining >> (numBitsRemaining - number)
                streamRemaining = streamRemaining - (subStream << (numBitsRemaining - number))
                numBitsRemaining = numBitsRemaining - number
                
                returnStream, returnNumBitsRemaining, returnVersion = processPacketsNumBits(subStream, number)
                if (returnStream != 0 or returnNumBitsRemaining != 0):
                    print("Error: We expected to process all {} bits of packet, but still have {} bits unprocessed".format(number, returnNumBitsRemaining))                
                versionTotal += returnVersion
            else:
                streamRemaining, numBitsRemaining, returnVersion = processPacketsNumPackets(streamRemaining, number, numBitsRemaining)
                versionTotal += returnVersion

    return streamRemaining, numBitsRemaining, versionTotal,

#Processes stream up to number of packets to process and returns the remaining stream
def processPacketsNumPackets(streamOrig, numPacketsToProcess, numBitsOrig):
    versionTotal = 0
    N = numPacketsToProcess
    streamRemaining = streamOrig
    numBitsRemaining = numBitsOrig

    while N>0:        
        version, typeID, lengthType, number, streamRemaining, numBitsRemaining = readPacketData(streamRemaining, numBitsRemaining)
        print("Read type numpackets packet with version {}, typeID {}, lengthType {}, number {}, and remainingBits {}".format(version, typeID, lengthType, number, numBitsRemaining))        
        versionTotal += version
        #if version == 0:
        #    print("version error, zero version is not allowed")
        #    return
       
        #Handle number
        if typeID ==4:
            N -= 1
            continue
        #Handle operator where we know the number of bits of the packets
        else:
            if lengthType == 0:
                subStream = streamRemaining >> (numBitsRemaining - number)
                streamRemaining = streamRemaining - (subStream << (numBitsRemaining - number))
                numBitsRemaining = numBitsRemaining - number
                
                returnStream, returnNumBitsRemaining, returnVersion = processPacketsNumBits(subStream, number)
                if (returnStream != 0 or returnNumBitsRemaining != 0):
                    print("Warning: We expected to process all {} bits of packet, but still have {} bits unprocessed".format(number, returnNumBitsRemaining))
                versionTotal += returnVersion
            else:
                streamRemaining, numBitsRemaining, returnVersion = processPacketsNumPackets(streamRemaining, number, numBitsRemaining)
                versionTotal += returnVersion
        N -= 1
    return streamRemaining, numBitsRemaining, versionTotal
    

def processInputString(value):
    stream = int(value, 16)
    numBits = int(len(value)*4)
    print("Starting off with a packet of {} bits".format(numBits))
    _, _, total = processPacketsNumBits(stream, numBits)
    return total

def testReadPacketData(value):
    stream = int(value, 16)
    numBits = int(len(value)*4)
    version, typeID, lengthType, number, streamRemaining, numBitsRemaining = readPacketData(stream, numBits)
    print("Read type numbits packet with version {}, typeID {}, lengthType {}, number {}, and remainingBits {}".format(version, typeID, lengthType, number, numBitsRemaining))
    return

    
#input = "D2FE28"
#testReadPacketData("D2FE28")
#processInputString("D2FE28")
#processInputString("38006F45291200")
#processInputString("EE00D40C823060")
#processInputString("8A004A801A8002F478")
#processInputString("620080001611562C8802118E34")
processInputString("C0015000016115A2E0802F182340")
#processInputString(input)

#print(f"{hexInput:032b}")

Starting off with a packet of 112 bits
Read type numbits packet with version 6, typeID 0, lengthType 0, number 84, and remainingBits 90
Read type numbits packet with version 0, typeID 0, lengthType 0, number 22, and remainingBits 62
Read type numbits packet with version 0, typeID 4, lengthType -1, number 10, and remainingBits 11
Read type numbits packet with version 6, typeID 4, lengthType -1, number 11, and remainingBits 0
Read type numbits packet with version 4, typeID 0, lengthType 1, number 2, and remainingBits 22
Read type numpackets packet with version 7, typeID 4, lengthType -1, number 12, and remainingBits 11
Read type numpackets packet with version 0, typeID 4, lengthType -1, number 13, and remainingBits 0


23

In [109]:
import numpy as np

#Part 2 is mostly same as day 1, except we add operator values
#NOTE: WE FIX A BUG ON PART 2 WHERE WE EXITED EARLY IF STREAM WAS 0

#Returns popped value, remaining stream, and number of bits in remaining stream
def popLeftN(stream, size, n):
    #Pad if we need to pop more than size bits
    if n > size:
        print("Warning: Padding extra bits")
        stream << (n-size)
        size = n
    
    mask = 2**n -1
    shift = size - n
    leftN = (stream >> shift) & mask
    remainder = stream - (leftN << shift)
    return leftN, remainder, shift


def readPacketData(streamOrig, numBits):
    #Get version
    version, stream, numBits = popLeftN(streamOrig, numBits, 3)
    
    #Get typeID
    typeID, stream, numBits = popLeftN(stream, numBits, 3)
    
    number = 0
    lengthTypeID = -1
    #Check literal value
    if typeID == 4:
        foundEnd = False
        while (not foundEnd):
            bits, stream, numBits = popLeftN(stream, numBits, 5)
            foundEnd = (bits >> 4) ^ 1
            number = (number << 4) | (bits & 0xF)
            #print("Have number {} and foundEnd is {} from bits {}".format(number, foundEnd, bits))
        if (not foundEnd):
            print("Warning, terminating but did not find number")
        if (not foundEnd) and stream != 0:
            print("Error: Did not find end of number and stream remains. Please investigate")
            return 0, 0, 0, 0, 0, 0
    else:
        lengthTypeID, stream, numBits = popLeftN(stream, numBits, 1)
        
    #Check length type 0
    if lengthTypeID == 0:
        number, stream, numBits = popLeftN(stream, numBits, 15)
    #Check length type 1:
    if lengthTypeID ==1:
        number, stream, numBits = popLeftN(stream, numBits, 11)
        
    return version, typeID, lengthTypeID, number, stream, numBits


#Processes entirety of stream up to numBitsRemaining
#This is an add version operator
def processPacketsNumBits(streamOrig, numBitsOrig):
    streamRemaining = streamOrig
    numBitsRemaining = numBitsOrig
    minPacketSize = 11
    values = []
    
    while numBitsRemaining > 0:
        if numBitsRemaining < minPacketSize:
            print("Warning: Ignoring the last extra {} bits of packet".format(numBitsRemaining))
            break
        
        #number is valid of the number or the return of number of bits or number of packets.
        #stream is the remaining stream after the header is read
        #numBits is the number of bits remaining of the original stream
        version, typeID, lengthType, number, streamRemaining, numBitsRemaining = readPacketData(streamRemaining, numBitsRemaining)        
        print("Read type numbits packet with version {}, typeID {}, lengthType {}, number {}, and remainingBits {}".format(version, typeID, lengthType, number, numBitsRemaining))
        
        #Handle number
        if typeID ==4:
            values.append(number)
            continue
        #Handle operator where we know the number of bits of the packets
        else:
            if lengthType == 0:
                subStream = streamRemaining >> (numBitsRemaining - number)
                streamRemaining = streamRemaining - (subStream << (numBitsRemaining - number))
                numBitsRemaining = numBitsRemaining - number
                
                returnStream, returnNumBitsRemaining, returnValues = processPacketsNumBits(subStream, number)
                if (returnStream != 0 or returnNumBitsRemaining != 0):
                    print("Error: We expected to process all {} bits of packet, but still have {} bits unprocessed".format(number, returnNumBitsRemaining))                
                values.append(processOperator(returnValues, typeID))
            else:
                streamRemaining, numBitsRemaining, returnValues = processPacketsNumPackets(streamRemaining, number, numBitsRemaining)
                values.append(processOperator(returnValues, typeID))

    return streamRemaining, numBitsRemaining, values

#Processes stream up to number of packets to process and returns the remaining stream
def processPacketsNumPackets(streamOrig, numPacketsToProcess, numBitsOrig):
    versionTotal = 0
    N = numPacketsToProcess
    streamRemaining = streamOrig
    numBitsRemaining = numBitsOrig

    values = []
    while N>0:        
        version, typeID, lengthType, number, streamRemaining, numBitsRemaining = readPacketData(streamRemaining, numBitsRemaining)
        print("Read type numpackets packet with version {}, typeID {}, lengthType {}, number {}, and remainingBits {}".format(version, typeID, lengthType, number, numBitsRemaining))        
       
        #Handle number
        if typeID ==4:
            values.append(number)
            N -= 1
            continue
        #Handle operator where we know the number of bits of the packets
        else:
            if lengthType == 0:
                subStream = streamRemaining >> (numBitsRemaining - number)
                streamRemaining = streamRemaining - (subStream << (numBitsRemaining - number))
                numBitsRemaining = numBitsRemaining - number
                
                returnStream, returnNumBitsRemaining, returnValues = processPacketsNumBits(subStream, number)
                if (returnStream != 0 or returnNumBitsRemaining != 0):
                    print("Warning: We expected to process all {} bits of packet, but still have {} bits unprocessed".format(number, returnNumBitsRemaining))
                values.append(processOperator(returnValues, typeID))
            else:
                streamRemaining, numBitsRemaining, returnValues = processPacketsNumPackets(streamRemaining, number, numBitsRemaining)
                values.append(processOperator(returnValues,typeID))
        N -= 1
    return streamRemaining, numBitsRemaining, values

def processOperator(values, operator):
    if operator == 0:
        return sum(values)
    if operator == 1:
        return np.prod(values)
    if operator == 2:
        return min(values)
    if operator == 3:
        return max(values)
    if operator == 5:
        if len(values) != 2:
            print("Error, invlid type 5 packet")
            exit(1)
        return 1 if values[0] > values[1] else 0
    if operator == 6:
        if len(values) != 2:
            print("Error, invlid type 6 packet")
            exit(1)
        return 1 if values[0] < values[1] else 0
    if operator == 7:
        if len(values) != 2:
            print("Error, invlid type 7 packet")
            exit(1)
        return 1 if values[0] == values[1] else 0
    
#processInputString("C200B40A82")
#processInputString("04005AC33890")
#processInputString("880086C3E88112")
#processInputString("CE00C43D881120")
#processInputString("D8005AC2A8F0")
#processInputString("F600BC2D8F")
#processInputString("9C005AC2F8F0")
processInputString("9C0141080250320F1802104A08")
#processInputString(input)

Starting off with a packet of 104 bits
Read type numbits packet with version 4, typeID 7, lengthType 0, number 80, and remainingBits 82
Read type numbits packet with version 2, typeID 0, lengthType 1, number 2, and remainingBits 62
Read type numpackets packet with version 2, typeID 4, lengthType -1, number 1, and remainingBits 51
Read type numpackets packet with version 4, typeID 4, lengthType -1, number 3, and remainingBits 40
Read type numbits packet with version 6, typeID 1, lengthType 1, number 2, and remainingBits 22
Read type numpackets packet with version 0, typeID 4, lengthType -1, number 2, and remainingBits 11
Read type numpackets packet with version 2, typeID 4, lengthType -1, number 2, and remainingBits 0


[1]